In [1]:
from IPython.display import Image, display
from ipywidgets import widgets,interact
import cantera as ct
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from collections import defaultdict
import os,shutil
%matplotlib inline

When using Cantera, the first thing you usually need is an object representing some phase of matter. Here, we'll create a gas mixture:

In [2]:
gas = ct.Solution('gri30.xml')

To view the state of the mixture, *call* the `gas` object as if it were a function:

In [3]:
gas()


  gri30:

       temperature             300  K
          pressure          101325  Pa
           density       0.0818891  kg/m^3
  mean mol. weight         2.01588  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy           26470        5.336e+04     J
   internal energy     -1.2109e+06       -2.441e+06     J
           entropy           64914        1.309e+05     J/K
    Gibbs function     -1.9448e+07        -3.92e+07     J
 heat capacity c_p           14312        2.885e+04     J/K
 heat capacity c_v           10187        2.054e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                H2              1                1         -15.7173
     [  +52 minor]              0                0



In [4]:
print (gas.species_names)

['H2', 'H', 'O', 'O2', 'OH', 'H2O', 'HO2', 'H2O2', 'C', 'CH', 'CH2', 'CH2(S)', 'CH3', 'CH4', 'CO', 'CO2', 'HCO', 'CH2O', 'CH2OH', 'CH3O', 'CH3OH', 'C2H', 'C2H2', 'C2H3', 'C2H4', 'C2H5', 'C2H6', 'HCCO', 'CH2CO', 'HCCOH', 'N', 'NH', 'NH2', 'NH3', 'NNH', 'NO', 'NO2', 'N2O', 'HNO', 'CN', 'HCN', 'H2CN', 'HCNN', 'HCNO', 'HOCN', 'HNCO', 'NCO', 'N2', 'AR', 'C3H7', 'C3H8', 'CH2CHO', 'CH3CHO']


## Shock tube ignition delay measurement conditions (Spadaccini, L.J., and Colket, M.B., (1994) Prog. Energy Combust. Sci. 20, 431. ) 
* $CH_4-C_2H_6-O_2-Ar$ (3.29%-0.21%-7%-89.5%)  
* phi = 1.045
* P = 6.1 - 7.6 atm
* T = 1356 - 1688 K 
* Set temperature,pressure and composition

In [5]:
gas.TPX = 1550., 6.5*ct.one_atm,'CH4:3.29, C2H6:0.21, O2:7 , Ar:89.5'

In [6]:
gas()


  gri30:

       temperature            1550  K
          pressure          658612  Pa
           density         1.97186  kg/m^3
  mean mol. weight         38.5843  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy      6.9211e+05         2.67e+07     J
   internal energy       3.581e+05        1.382e+07     J
           entropy          4803.4        1.853e+05     J/K
    Gibbs function     -6.7532e+06       -2.606e+08     J
 heat capacity c_p          634.94         2.45e+04     J/K
 heat capacity c_v          419.45        1.618e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                O2           0.07        0.0580525         -28.6786
               CH4         0.0329        0.0136793          -34.996
              C2H6         0.0021       0.00163658         -47.0076
                AR        

* Residence time is close to ignition delay reported by (Spadaccini, L.J., and Colket, M.B., (1994) Prog. Energy Combust. Sci. 20, 431. ) 

In [7]:
residence_time = 1e-3

* Create a batch reactor object and set solver tolerances

In [8]:
reactor = ct.IdealGasConstPressureReactor(gas, energy='on')
reactor_network = ct.ReactorNet([reactor])
reactor_network.atol = 1e-12
reactor_network.rtol = 1e-12

* Store time, pressure, temperature and mole fractions

In [9]:
folders =['dot_files','png_files','pdf_files']
for folder in folders:
    if os.path.isdir(folder):
        pass
    else:
        os.makedirs(folder)

In [10]:
profiles = defaultdict(list)
time =0
steps = 0
while time < residence_time:
    profiles['time'].append(time)
    profiles['pressure'].append(gas.P)
    profiles['temperature'].append(gas.T)
    profiles['mole_fractions'].append(gas.X)
    time = reactor_network.step()
    steps = +1


# Interactive reaction path diagram

In [21]:
@interact( plot_step=widgets.IntSlider(value=100,min=0,max=steps,step=10),
           threshold=widgets.FloatSlider(value=0.005,min=0,max=1),
         details = widgets.ToggleButton(),
         species = widgets.Dropdown(options=gas.element_names,
    value='C',
    description='Element',
    disabled=False,))
def plot_reaction_path_diagrams(plot_step, threshold,details,species):
    P = profiles['pressure'][plot_step]
    T = profiles['temperature'][plot_step]
    X = profiles['mole_fractions'][plot_step]
    time= profiles['time'][plot_step]
    gas.TPX = T,P,X
    print("time = {:.2g} s".format(time))
    
    diagram = ct.ReactionPathDiagram(gas, species)
    diagram.threshold = threshold

    diagram.show_details = details
    dot_file = 'reaction_paths.dot'
    png_file = 'reactio_paths.png'
    diagram.write_dot(dot_file)
    os.system('dot {0} -Tpng -o{1} -Gdpi=100'.format(dot_file, png_file))
    img = Image(filename=png_file)
    display(img)
    

A Jupyter Widget

* Find reactions containing the specy of interest 

In [13]:
C2H6_stoichiometry = np.zeros_like(gas.reactions())
for i, r in enumerate(gas.reactions()):
    C2H6_moles = r.products.get('C2H6',0) - r.reactants.get('C2H6',0)
    C2H6_stoichiometry[i] = C2H6_moles
C2H6_reaction_indices = C2H6_stoichiometry.nonzero()[0]

### The following cell calculates net rates of progress of reactions containing the species of interest and stores it

In [14]:
profiles['C2H6_production_rates'] = []
for i in range(len(profiles['time'])):
    X = profiles['mole_fractions'][i]
    t = profiles['time'][i]
    T = profiles['temperature'][i]
    P = profiles['pressure'][i]
    gas.TPX = (T, P, X)
    C2H6_production_rates = (gas.net_rates_of_progress * #  [kmol/m^3/s]
                           C2H6_stoichiometry *
                           gas.volume_mass  # Specific volume [m^3/kg].
                         ) # overall, mol/s/g  (g total in reactor, same basis as N_atoms_in_fuel)
    
    profiles['C2H6_production_rates'].append(C2H6_production_rates[C2H6_reaction_indices])


### Plot the instantaneous fluxes

In [15]:
@interact(annotation_cutoff=widgets.FloatSlider(value=1e-2,min=1e-2,max=4,steps=10),
         profiles=widgets.fixed(profiles))
def plot_instantaneous_fluxes(profiles,annotation_cutoff):
    profiles = profiles
    fig = plt.figure(figsize=(6,6),dpi=100)
    plt.plot(
        profiles['time'],
        np.array(profiles['C2H6_production_rates']))
    #plt.xlim(5e-5,5e-2)
    #plt.ylim(-1e-2,3e-2)

    for i, C2H6_production_rate in enumerate(np.array(profiles['C2H6_production_rates']).T):
        peak_index = abs(C2H6_production_rate).argmax()
        peak_time = profiles['time'][peak_index]
        peak_C2H6_production = C2H6_production_rate[peak_index]
        reaction_string = gas.reaction_equations(C2H6_reaction_indices)[i]

        if abs(peak_C2H6_production)>annotation_cutoff:
            plt.annotate((reaction_string).replace('2','$_2$').replace('<=>','='),
                         xy=(peak_time, peak_C2H6_production),
                         xytext=(peak_time*2, (peak_C2H6_production +
                                               0.003*(peak_C2H6_production/abs(peak_C2H6_production)) * \
                                               (abs(peak_C2H6_production)>0.005) * \
                                               (peak_C2H6_production<0.06) )
                                              ),
                         arrowprops=dict(arrowstyle="->",
                                        color='black',
                                        relpos=(0,0.6),
                                        linewidth=2,),
                         horizontalalignment='left',
                        )


    plt.xlabel('Time (s)',fontsize=16)
    plt.ylabel('Net rates of C2H6 production',fontsize=16)
    plt.tight_layout()
    plt.show()

A Jupyter Widget

### Integrating over time using `scipy.integrate.cumptraz` 

In [16]:
from scipy import integrate
integrated_fluxes = integrate.cumtrapz(np.array(profiles['C2H6_production_rates']), 
                                             np.array(profiles['time']), 
                                             axis=0,
                                      initial=0)

### Plot  integrated fluxes

In [17]:
@interact(annotation_cutoff=widgets.FloatSlider(value=1e-6,min=1e-6,max=1e-4,steps=100),
         profiles=widgets.fixed(profiles),
         integrated_fluxes=widgets.fixed(integrated_fluxes))
def plot_integrated_fluxes(profiles,integrated_fluxes,annotation_cutoff):
    profiles = profiles
    integrated_fluxes = integrated_fluxes
    fig = plt.figure(figsize=(6,6),dpi=100)
    plt.plot(
    profiles['time'],
    integrated_fluxes)
    final_time = profiles['time'][-1]
    for i, C2H6_production in enumerate(integrated_fluxes.T):
        total_C2H6_production = C2H6_production[-1]
        reaction_string = gas.reaction_equations(C2H6_reaction_indices)[i]

        if abs(total_C2H6_production)>annotation_cutoff:
            plt.text(final_time*1.06, total_C2H6_production, reaction_string)

    plt.xlabel('Time (s)',fontsize=16)
    plt.ylabel('Integrated net rate of progress',fontsize=16)
    plt.title("Cumulative C2H6 formation in flowtube",fontsize=16)
    plt.tight_layout()
    #plt.savefig('plots/cumulative_NO_flux.pdf')
    plt.show()

A Jupyter Widget

In [ ]:
plot_integrated_fluxes(profiles=profiles,integrated_fluxes=integrated_fluxes,annotation_cutoff=1e-6)

In [14]:
@interact(annotation_cutoff=widgets.FloatSlider(value=1e-2),min=1e-8,max=1e2,steps=1e1)
plot_instantaneous_fluxes(profiles=profiles,annonation_cutoff=1e-2)

SyntaxError: invalid syntax (<ipython-input-14-602190c97fcb>, line 2)

In [16]:
widgets.fixed?

In [20]:
NO_reaction_indices

array([177, 178, 179, 181, 185, 186, 187, 188, 189, 194, 197, 198, 207,
       211, 212, 213, 214, 215, 221, 223, 225, 227, 228, 243, 244, 245,
       246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 257, 273, 280, 282])

1461

In [8]:
print (gas.reaction_equation)

<built-in method reaction_equation of Solution object at 0x114383b38>
